<a href="https://colab.research.google.com/github/Elwing-Chou/ml0716/blob/master/news.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
import zipfile
f = zipfile.ZipFile("drive/My Drive/0724test/chinese_news_trans.zip")
f.extractall("train")
f = zipfile.ZipFile("drive/My Drive/0724test/chinese_news_test.zip")
f.extractall("test")

In [11]:
import glob
import os
import pandas as pd

def getdf(targetdn):
    datas = {
        "path":[],
        "label":[]
    }
    base = ""
    p = os.path.join(base, targetdn, "*")
    for dn in glob.glob(p):
        name = os.path.split(dn)[-1]
        fn = os.path.join(dn, "*.[tT][xX][tT]")
        fns = glob.glob(fn)
        datas["label"] = datas["label"] + [name] * len(fns)
        datas["path"] = datas["path"] + fns
    return pd.DataFrame(datas)

In [12]:
train_df = getdf("train/chinese_news_trans")
test_df = getdf("test/chinese_news_test")

In [14]:
def read_content(fn):
    with open(fn, "r", encoding="utf-8") as f:
        content = f.read()
    return content
train_df["content"] = train_df["path"].apply(read_content)
test_df["content"] = test_df["path"].apply(read_content)

In [16]:
test_df

,path,label,content
0,test/chinese_news_test/軍事/8240.txt,軍事,美海軍成功演示飛機在空中控制無人機(附圖) \n\n--------------------...
1,test/chinese_news_test/軍事/8249.txt,軍事,\n軍事評論：“巴格達之戰”考驗美軍巷戰武器裝備(2) \n\n--------------...
2,test/chinese_news_test/軍事/8245.txt,軍事,軍事評論：美英聯軍地面作戰裝備特點分析(2) \n\n-------------------...
3,test/chinese_news_test/軍事/8244.txt,軍事,在伊拉克戰爭中使用的空襲精確制導彈藥(1) \n\n--------------------...
4,test/chinese_news_test/軍事/8247.txt,軍事,軍事評論：從巷戰或地道戰看單兵夜視裝備的發展 \n\n-------------------...
...,...,...,...
96,test/chinese_news_test/政治/9505.TXT,政治,楊主席提出促進中拉關係的５項建議\n新華社蒙得維的亞５月２３日電正在這...
97,test/chinese_news_test/政治/9501.TXT,政治,阿拉伯外長會議閉幕\n新華社巴格達５月２４日電為期２天的阿拉伯外長會\...
98,test/chinese_news_test/政治/9500.TXT,政治,外事簡訊\n新華社北京５月２４日電外事簡訊\n 全國人大常委會副...
99,test/chinese_news_test/政治/9497.TXT,政治,第３次安第斯集團首腦會議閉幕\n新華社庫斯科（祕魯）５月２３日電第３次...
